# Worcester EV Charging Sites - Exploratory Data Analysis

**MA EV ChargeMap Portfolio Project**

This notebook performs exploratory data analysis on the Worcester, MA candidate EV charging sites.

## Goals
1. Understand the distribution of features across sites
2. Explore relationships between features
3. Analyze computed scores
4. Identify high-opportunity locations
5. Visualize spatial patterns

In [ ]:
# Import libraries
import sys
sys.path.append('../backend')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

%matplotlib inline

## 1. Load Data

In [ ]:
# Connect to database
from app.config import settings

engine = create_engine(settings.database_url)

# Load sites data
df = pd.read_sql("SELECT * FROM sites WHERE city = 'worcester'", engine)

print(f"Loaded {len(df)} sites")
df.head()

In [ ]:
# Check for missing values
print("Missing values:")
print(df.isnull().sum())

print("\nData types:")
print(df.dtypes)

## 2. Feature Distributions

Let's examine the distribution of input features.

In [ ]:
# Feature columns
feature_cols = ['traffic_index', 'pop_density_index', 'renters_share', 
                'income_index', 'poi_index']

# Plot distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for idx, col in enumerate(feature_cols):
    axes[idx].hist(df[col], bins=30, edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'{col.replace("_", " ").title()}')
    axes[idx].set_xlabel('Value')
    axes[idx].set_ylabel('Frequency')
    axes[idx].axvline(df[col].mean(), color='red', linestyle='--', label='Mean')
    axes[idx].legend()

# Hide last subplot
axes[-1].axis('off')

plt.tight_layout()
plt.suptitle('Feature Distributions', y=1.02, fontsize=16, fontweight='bold')
plt.show()

In [ ]:
# Summary statistics
print("Feature Summary Statistics:")
df[feature_cols].describe()

## 3. Score Distributions

In [ ]:
# Score columns
score_cols = ['score_demand', 'score_equity', 'score_traffic', 'score_grid', 'score_overall']

# Plot score distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for idx, col in enumerate(score_cols):
    axes[idx].hist(df[col], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    axes[idx].set_title(f'{col.replace("score_", "").replace("_", " ").title()} Score')
    axes[idx].set_xlabel('Score (0-100)')
    axes[idx].set_ylabel('Frequency')
    axes[idx].axvline(df[col].mean(), color='red', linestyle='--', label=f'Mean: {df[col].mean():.1f}')
    axes[idx].legend()

# Hide last subplot
axes[-1].axis('off')

plt.tight_layout()
plt.suptitle('Score Distributions', y=1.02, fontsize=16, fontweight='bold')
plt.show()

In [ ]:
# Score summary
print("Score Summary Statistics:")
df[score_cols].describe()

## 4. Feature Correlations

In [ ]:
# Correlation matrix
correlation_cols = feature_cols + score_cols
corr_matrix = df[correlation_cols].corr()

# Plot heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Feature and Score Correlations', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Spatial Visualization

In [ ]:
# Scatter plot of sites colored by overall score
plt.figure(figsize=(14, 10))

scatter = plt.scatter(df['lng'], df['lat'], c=df['score_overall'], 
                     cmap='RdYlGn', s=50, alpha=0.6, edgecolors='black', linewidth=0.5)

plt.colorbar(scatter, label='Overall Score')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Worcester EV Charging Site Scores (Geographic Distribution)', 
          fontsize=16, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Top Opportunity Sites

In [ ]:
# Top 20 sites by overall score
top_sites = df.nlargest(20, 'score_overall')[[
    'id', 'location_label', 'score_overall', 'score_demand', 
    'score_equity', 'daily_kwh_estimate'
]]

print("Top 20 Sites by Overall Opportunity Score:")
print(top_sites.to_string(index=False))

In [ ]:
# Bar chart of top 10 sites
top_10 = df.nlargest(10, 'score_overall')

fig, ax = plt.subplots(figsize=(12, 6))

bars = ax.barh(range(len(top_10)), top_10['score_overall'], color='steelblue')
ax.set_yticks(range(len(top_10)))
ax.set_yticklabels([f"Site {id}" for id in top_10['id']])
ax.set_xlabel('Overall Score')
ax.set_title('Top 10 Sites by Overall Score', fontsize=14, fontweight='bold')
ax.invert_yaxis()

# Add value labels
for i, (bar, val) in enumerate(zip(bars, top_10['score_overall'])):
    ax.text(val + 1, i, f'{val:.1f}', va='center')

plt.tight_layout()
plt.show()

## 7. Demand Analysis

In [ ]:
# Daily kWh distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df['daily_kwh_estimate'], bins=30, edgecolor='black', alpha=0.7, color='orange')
plt.xlabel('Daily kWh Estimate')
plt.ylabel('Frequency')
plt.title('Distribution of Estimated Daily Charging Demand')
plt.axvline(df['daily_kwh_estimate'].mean(), color='red', linestyle='--', 
           label=f'Mean: {df["daily_kwh_estimate"].mean():.1f} kWh')
plt.legend()

plt.subplot(1, 2, 2)
plt.scatter(df['score_overall'], df['daily_kwh_estimate'], alpha=0.5)
plt.xlabel('Overall Score')
plt.ylabel('Daily kWh Estimate')
plt.title('Score vs. Estimated Demand')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Total estimated daily demand: {df['daily_kwh_estimate'].sum():,.0f} kWh/day")
print(f"Average per site: {df['daily_kwh_estimate'].mean():.1f} kWh/day")

## 8. Equity Analysis

In [ ]:
# Compare high equity vs high demand sites
high_equity = df[df['score_equity'] > df['score_equity'].quantile(0.75)]
high_demand = df[df['score_demand'] > df['score_demand'].quantile(0.75)]

print(f"High Equity Sites (top 25%): {len(high_equity)}")
print(f"High Demand Sites (top 25%): {len(high_demand)}")
print(f"Sites that are both high equity AND high demand: {len(set(high_equity.index) & set(high_demand.index))}")

# Scatter plot
plt.figure(figsize=(10, 8))
plt.scatter(df['score_demand'], df['score_equity'], 
           c=df['score_overall'], cmap='viridis', s=50, alpha=0.6)
plt.colorbar(label='Overall Score')
plt.xlabel('Demand Score')
plt.ylabel('Equity Score')
plt.title('Demand vs. Equity (colored by Overall Score)', fontsize=14, fontweight='bold')
plt.axhline(df['score_equity'].median(), color='red', linestyle='--', alpha=0.5, label='Median Equity')
plt.axvline(df['score_demand'].median(), color='blue', linestyle='--', alpha=0.5, label='Median Demand')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Key Insights

### Summary Statistics

Based on the analysis above, we can draw several conclusions:

1. **Feature Distributions**: Features show reasonable variation across the city, with some spatial clustering
2. **Score Correlations**: Scores are appropriately correlated with their input features
3. **High-Opportunity Sites**: Top sites balance demand, equity, and infrastructure readiness
4. **Demand Potential**: Total estimated charging demand provides insights into market size
5. **Equity Considerations**: Many high-opportunity sites serve underserved communities

### Next Steps

1. Train ML model on this data (see `02_model_training.ipynb`)
2. Validate model predictions
3. Deploy model in API for real-time scoring

In [ ]:
# Export summary for documentation
summary = {
    'total_sites': len(df),
    'mean_overall_score': df['score_overall'].mean(),
    'total_daily_kwh': df['daily_kwh_estimate'].sum(),
    'top_site_id': df.nlargest(1, 'score_overall')['id'].values[0],
    'top_site_score': df.nlargest(1, 'score_overall')['score_overall'].values[0],
}

print("\n=== Analysis Summary ===")
for key, value in summary.items():
    print(f"{key}: {value}")